In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing
from sklearn import model_selection

In [3]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

raw_train_df = train_df.copy()
raw_test_df = test_df.copy()

In [4]:
price = np.log(train_df.pop('SalePrice')+1)
all_df = pd.concat([train_df, test_df])

all_dummies = pd.get_dummies(all_df)
all_dummies.columns

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=289)

In [12]:
number_columns = all_dummies.columns[(all_dummies.dtypes != 'flat64').values]

In [23]:
print(all_dummies.dtypes)
all_df.dtypes

Id                         int64
MSSubClass                 int64
LotFrontage              float64
LotArea                    int64
OverallQual                int64
OverallCond                int64
YearBuilt                  int64
YearRemodAdd               int64
MasVnrArea               float64
BsmtFinSF1               float64
BsmtFinSF2               float64
BsmtUnfSF                float64
TotalBsmtSF              float64
1stFlrSF                   int64
2ndFlrSF                   int64
LowQualFinSF               int64
GrLivArea                  int64
BsmtFullBath             float64
BsmtHalfBath             float64
FullBath                   int64
HalfBath                   int64
BedroomAbvGr               int64
KitchenAbvGr               int64
TotRmsAbvGrd               int64
Fireplaces                 int64
GarageYrBlt              float64
GarageCars               float64
GarageArea               float64
WoodDeckSF                 int64
OpenPorchSF                int64
          

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
                  ...   
HalfBath           int64
BedroomAbvGr       int64
KitchenAbvGr       int64
KitchenQual       object
TotRmsAbvGrd       int64
Functional        object
Fireplaces         int64
FireplaceQu       object
GarageType        object


In [22]:
print(type(raw_train_df.isnull().sum()))
Ser = raw_train_df.isnull().sum().sort_values(ascending=False)
print(Ser)
print(Ser.index[Ser.values!=0])
notnull_column = Ser.index[Ser.values!=0]
print(all_df[notnull_column].dtypes)

<class 'pandas.core.series.Series'>
PoolQC           1453
MiscFeature      1406
Alley            1369
Fence            1179
FireplaceQu       690
LotFrontage       259
GarageCond         81
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageQual         81
BsmtExposure       38
BsmtFinType2       38
BsmtFinType1       37
BsmtCond           37
BsmtQual           37
MasVnrArea          8
MasVnrType          8
Electrical          1
Utilities           0
YearRemodAdd        0
MSSubClass          0
Foundation          0
ExterCond           0
ExterQual           0
Exterior2nd         0
Exterior1st         0
RoofMatl            0
RoofStyle           0
YearBuilt           0
                 ... 
GarageArea          0
PavedDrive          0
WoodDeckSF          0
OpenPorchSF         0
3SsnPorch           0
BsmtUnfSF           0
ScreenPorch         0
PoolArea            0
MiscVal             0
MoSold              0
YrSold              0
SaleType            0
Functional        

In [26]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import regression
import matplotlib.pyplot as plt


train_df = pd.read_csv('data/train.csv', index_col=0)  #直接把Id当成索引了
test_df = pd.read_csv('data/test.csv', index_col=0)

raw_train_df = train_df.copy()
raw_test_df = test_df.copy()

price = np.log(train_df.pop('SalePrice')+1)

all_df = pd.concat([train_df, test_df])
discard_columns = ['Alley', 'PoolQC', 'MiscFeature', 'Fence', 'FireplaceQu']
all_df.drop(all_df[discard_columns], axis=1, inplace=True)


all_dummies = pd.get_dummies(all_df)

number_columns = all_df.columns[(all_df.dtypes != 'object').values]

mean_series = all_dummies.mean(axis=0)

all_dummies.fillna(mean_series, axis=0, inplace=True)

#没有缺失数据了

scaler = preprocessing.StandardScaler()

# para = scaler.fit((all_dummies.loc[:, number_columns]))
scaled_df = scaler.fit_transform(all_dummies.loc[:, number_columns])
all_dummies.loc[:, number_columns] = scaled_df
# number_columns.

train_df, test_df = all_dummies.loc[train_df.index], all_dummies.loc[test_df.index]
xgb_model = xgb.XGBRegressor()

x_train = train_df.values
y_test = test_df.values

para_ = {'max_depth' : [2, 3, 4, 5], 'learning_rate':[0.01, 0.1, 1], 'n_estimators':[50, 100, 150, 200]}
para_search_model = model_selection.GridSearchCV(xgb_model, param_grid=para_, scoring='neg_mean_squared_error')
para_search_model.fit(x_train, price)

GridSearchCV(cv=None, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [2, 3, 4, 5], 'learning_rate': [0.01, 0.1, 1], 'n_estimators': [50, 100, 150, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [31]:
para_search_model.cv_results_

{'mean_fit_time': array([ 0.16166679,  0.38666662,  0.57233342,  0.77033337,  0.16033324,
         0.36133321,  0.63166666,  0.92933321,  0.15866653,  0.36833342,
         0.63366675,  0.95766671,  0.15833338,  0.36199999,  0.61733325,
         0.97400014,  0.20799979,  0.41166679,  0.61333354,  0.81599998,
         0.27499994,  0.56366666,  0.85199992,  1.14300005,  0.3343335 ,
         0.71266691,  1.08866652,  1.47633338,  0.38999987,  0.85966667,
         1.32733329,  1.79033335,  0.21233336,  0.4156665 ,  0.61666663,
         0.82000009,  0.29766663,  0.62166667,  0.89100011,  1.18766658,
         0.38833332,  0.82233334,  1.32466666,  1.49500004,  0.52600002,
         1.01533326,  1.13733331,  1.26833328]),
 'mean_score_time': array([ 0.00299994,  0.00300002,  0.0030001 ,  0.00333333,  0.00300002,
         0.00266663,  0.00299994,  0.00366672,  0.00300002,  0.00333333,
         0.00333325,  0.00366672,  0.00233332,  0.00300018,  0.0030001 ,
         0.00366656,  0.00333341,  0.00

In [27]:
para_search_model.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_learning_rate', 'param_max_depth', 'param_n_estimators', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'mean_train_score', 'std_train_score'])

In [34]:
print(para_search_model.best_params_)
para_search_model.best_index_

{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 200}


19

In [32]:
print(para_search_model.cv_results_['mean_test_score'].shape)
para_search_model.cv_results_['mean_test_score']

(48,)


array([ -4.88162333e+01,  -1.79528057e+01,  -6.62455716e+00,
        -2.45882312e+00,  -4.88162333e+01,  -1.79528057e+01,
        -6.62445196e+00,  -2.45807898e+00,  -4.88162333e+01,
        -1.79528057e+01,  -6.62448041e+00,  -2.45826364e+00,
        -4.88162333e+01,  -1.79528057e+01,  -6.62448041e+00,
        -2.45843132e+00,  -2.71288428e-02,  -1.82537271e-02,
        -1.68552790e-02,  -1.62617599e-02,  -2.48320662e-02,
        -1.74993851e-02,  -1.68165483e-02,  -1.64895180e-02,
        -2.37229660e-02,  -1.71033829e-02,  -1.66285537e-02,
        -1.64737725e-02,  -2.36815310e-02,  -1.72713864e-02,
        -1.68596809e-02,  -1.67030349e-02,  -2.63273777e-02,
        -2.50701618e-02,  -2.48808432e-02,  -2.48289899e-02,
        -3.27075729e-02,  -3.26951584e-02,  -3.29934825e-02,
        -3.29058518e-02,  -3.67461251e-02,  -3.69419224e-02,
        -3.68850895e-02,  -3.68908524e-02,  -3.90863868e-02,
        -3.90784033e-02,  -3.90784033e-02,  -3.90784033e-02])

In [36]:
np.sqrt(-(para_search_model.cv_results_['mean_test_score'][para_search_model.best_index_]))

0.12752160557878339

In [38]:
np.expm1(para_search_model.predict(y_test))

array([ 119749.5078125,  155981.421875 ,  184037.625    , ...,
        160849.359375 ,  119770.8671875,  232584.171875 ], dtype=float32)

In [40]:
test_df.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=270)

In [45]:
xgb_submission = pd.DataFrame({'Id':test_df.index, 'SalePrice':np.expm1(para_search_model.predict(y_test))})
xgb_submission.to_csv('output/xgb_submission.csv', index=False)

In [3]:
c1 = pd.read_csv('input/train.csv', nrows=2)

In [4]:
c1

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [7]:
c2=pd.read_csv('input/train.csv', nrows=2, skiprows=2)

In [8]:
c2

,2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,...,0.10,NA.1,NA.2,NA.3,0.11,5,2007,WD,Normal,181500
0,3,60,RL,68,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
1,4,70,RL,60,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000


In [29]:
import numpy as np
a = np.arange(1,19)
a = np.reshape(a,(2,3,3))

In [13]:
a
b = a.tolist()
b

[[[1, 2, 3], [4, 5, 6], [7, 8, 9]], [[10, 11, 12], [13, 14, 15], [16, 17, 18]]]

In [19]:
import pandas as pd
aa = pd.DataFrame(b)

In [22]:
aa['lable'] = [[4],[6]]

In [23]:
aa

,0,1,2,lable
0,"[1, 2, 3]","[4, 5, 6]","[7, 8, 9]",[4]
1,"[10, 11, 12]","[13, 14, 15]","[16, 17, 18]",[6]


In [24]:
aa.values

array([[list([1, 2, 3]), list([4, 5, 6]), list([7, 8, 9]), list([4])],
       [list([10, 11, 12]), list([13, 14, 15]), list([16, 17, 18]),
        list([6])]], dtype=object)

In [25]:
aa.values[:, -1]

array([list([4]), list([6])], dtype=object)

In [26]:
aa.values[:, :-1]

array([[list([1, 2, 3]), list([4, 5, 6]), list([7, 8, 9])],
       [list([10, 11, 12]), list([13, 14, 15]), list([16, 17, 18])]], dtype=object)

In [27]:
aa.shape

(2, 4)

In [31]:
[x for x in aa.values]

[array([list([1, 2, 3]), list([4, 5, 6]), list([7, 8, 9]), list([4])], dtype=object),
 array([list([10, 11, 12]), list([13, 14, 15]), list([16, 17, 18]),
        list([6])], dtype=object)]

array([[[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9]],

       [[10, 11, 12],
        [13, 14, 15],
        [16, 17, 18]]])

In [34]:
a = np.ones((3,3,3))

In [35]:
a

array([[[ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.]],

       [[ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.]],

       [[ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.]]])

In [36]:
a.shape

(3, 3, 3)

In [38]:
a[1, :, :].shape

(3, 3)